In [ ]:
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.5 MB 1.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import random

In [ ]:
orders = pd.read_csv('orders.csv')
couriers = pd.read_csv('partners_delays.csv')

In [ ]:
orders['date'] = pd.to_datetime(orders['date'])
couriers['dttm'] = pd.to_datetime(couriers['dttm'])

### Что тут происходит? Мы ожидаем получить датасет с датой, зонами доставки, количеством курьеров и количеством заказов. На выходе мы получаем датасет, в котором столбцы имеют вид data_area, где data это день без часов(2021-04-01), а area это зона доставки. Еще у нас есть столбец time, в котором находятся числа от 8 до 23, это часы доставки, там будет написано, сколько курьеров нам нужно.

In [ ]:
def optimize(dataset):
  index = 0
  work_shifts = [3, 4, 5, 6, 7]
  dict_list = []
  
  for area in dataset['delivery_area_id'].unique():
    for i in range(7):
      start_date = dataset['dttm'].loc[index]
      end_date = (dataset[(dataset['delivery_area_id'] == area) & (dataset['dttm'] < start_date + pd.DateOffset(days=1))]['dttm'].index)[-1]
      ord_count = dataset['orders'][index:end_date + 1].sum()
      dates = {}
      for i in range(index, end_date+1):
        dates[str(new.loc[i]['dttm'])] = new.loc[i]['orders']

      max_workers = int(new['partners_cnt'][index:end_date+1].max())

      worker = {}
      for i in range(max_workers):
        worker['{}'.format(i)] = 1
      workers, pay = gp.multidict(worker)

      c = 0
      availability = gp.tuplelist([])
      for i in range(index, end_date+1):
        for k in range(int(new.loc[i]['partners_cnt'])):
          if c == max_workers:
            c = 0
          availability.append(('{}'.format(workers[c]),'{}'.format(new.loc[i]['dttm'])))
          c+= 1

      index = end_date + 1
      shifts, shiftRequirements = gp.multidict(dates)

      m = gp.Model("workforce5")
      x = m.addVars(availability, vtype=GRB.BINARY, name="x")
      slacks = m.addVars(shifts, name="Slack")
      totSlack = m.addVar(name='totSlack')
      totShifts = m.addVars(workers, name="TotShifts")
      shift_reqmts = m.addConstrs((x.sum('*',s) + slacks[s] == shiftRequirements[s] for s in shifts), name='shiftRequirement')
      num_temps = m.addConstr(totSlack == slacks.sum(), name='totSlack')
      num_shifts = m.addConstrs((totShifts[w] == x.sum(w,'*') for w in workers), name='totShifts')
      minShift = m.addVar(4, name='minShift')
      maxShift = m.addVar(8,name='maxShift')
      min_constr = m.addGenConstrMin(minShift, totShifts, name='minShift')
      """ Если убрать комментари со следующей строки кода, то мы получим ограничение на макс. продолжительность смены 8 часов,
          Но существуют такие дни, когда это условие не может выполниться, курьеров значительно меньше, чем заказов, и либо кто-то
          Не получает свой заказ, либо кто-то работает больше 8 часов(Я так это понял)
      """ 
      #max_constr = m.addGenConstrMin(maxShift, totShifts, name='maxShift')
      m.ModelSense = GRB.MINIMIZE
      m.setObjectiveN(totSlack, index=0, priority=2, reltol=0.2, name='TotalSlack')
      m.setObjectiveN(maxShift - minShift, index=1, priority=1, name='Fairness')
      m.write('workforce.lp')
      m.optimize()
      # The Status attribute  provides current optimization status of the model object m
      # In workforce model, we check if the model is infeasible or unbounded and report this situation
      status = m.Status
      if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE  or status == GRB.Status.UNBOUNDED:
          print('The model cannot be solved because it is infeasible or unbounded')
          sys.exit(0)
      # If the optimization status of the model is not optimal for some other reason, we report that 
      # situation.
      if status != GRB.Status.OPTIMAL:
          print('Optimization was stopped with status ' + str(status))
          sys.exit(0)

      # Print total slack and the number of shifts worked for each worker
      # The KPIs for this optimization number is the number of extra worked required to satisfy 
      # demand and the number of shifts that each employed worker is working.
      solution = {}
      shifts_sol = {}
      solution['Total slack required'] = str(totSlack.X)
      assignments_all = {}
      gant={}

      assignments = dict()
      for [w, s] in availability:
          if x[w, s].x == 1:
              if w in assignments:
                  assignments[w].append(s)
              else:
                  assignments[w] = [s]

      """
      Уберите комментарии если хотите увидеть графики распределения смен
      """
      #print(pd.DataFrame.from_records(list(solution.items()), columns=['KPI', 'Value']))
      #print('-'*50)

      for w in workers:
          shifts_sol[w]=totShifts[w].X
          assignments_all[w]=assignments.get(w, [])

      #print('Shifts')
      #print(pd.DataFrame.from_records(list(shifts_sol.items()), columns=['Worker', 'Number of shifts']))
          
      y_pos = np.arange(len(shifts_sol.keys())) 
      #plt.bar(y_pos,shifts_sol.values() , align='center')
      #plt.xticks(y_pos, shifts_sol.keys())
      #plt.show()

      #print('-'*50)
      for w in assignments_all:
          gant[w] = [w]
          for d in shifts:
              gant[w].append('*' if d in assignments_all[w] else '-')

      #print('Assigments')
      #print('Symbols: \'-\': not working, \'*\': working')
      #pd.set_option('display.width', 1000)
      #print(pd.DataFrame.from_records(list(gant.values()), columns=['worker']+shifts))
      #print(pd.DataFrame.from_records(list(gant.values())))
      df['{}'.format(str(start_date)[:10])+"_"+str(area)] = 0
      for i in range(len(shifts)):
        cour = 0
        for k in workers:
          if gant[k][i +1 ] == '*':
            cour += 1

        ind = df[(df['time'] == pd.to_datetime(shifts[i]).hour)].index
        df['{}'.format(str(shifts[i])[:10])+"_"+str(area)][ind] = cour
  return df

In [ ]:
optimize(new)

Выходные данные были обрезаны до нескольких последних строк (5000).

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 12 rows and 11 columns
Presolve time: 0.00s
Presolved: 4 rows and 21 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (TotalSlack) ...
---------------------------------------------------------------------------

Presolve added 1 rows and 0 columns
Presolve removed 0 rows and 12 columns
Presolve time: 0.00s
Presolved: 5 rows, 9 columns, 15 nonzeros
Variable types: 3 continuous, 6 integer (4 binary)
Found heuristic solution: objective 55.0000000
Found heuristic solution: objective 54.0000000

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 2: 54 55 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.400000000